In [55]:
import psycopg2
import csv
import pandas as pd

conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                        user='rpa_robot', password='Qaz123123+')

cur = conn.cursor(name='1583_first_part')

start_date = '2023-04-30'
end_date = '2023-05-31'
store_id = '985'

query = f"""
select qfo.source_store_id,
	   ds.store_name,
       sum(qfo.cnt_wares_pos_detail)                                                             as "Кол-во товаров в чеках", -- "Количество товаров в чеке"
       sum(qfo.order_sum_with_vat)                                                               as "Выручка, тг с НДС",
       sum(qfo.order_sum_without_vat)                                                            as "Выручка, тг без НДС",
       sum(qfo.is_sale_order)                                                                    as "Количество чеков"
from dwh_data.qs_fact_order qfo
left join dwh_data.dim_store ds on ds.source_store_id = qfo.source_store_id and current_date between ds.datestart and ds.dateend
where date(qfo.order_date) between to_date('2023-04-30', 'YYYY-MM-DD') and to_date('2023-05-31', 'YYYY-MM-DD')
        and qfo."source_store_id"::int > 0 and ds."store_name" like '%Торговый%'
group by qfo.source_store_id, ds.store_name
     union all
select   qfog.source_store_id,
	   ds.store_name,
         sum(qfog.count_wares)                            as quantity,
         sum(qfog.sum_sale_vat)                           as order_sum_with_vat,
         sum(qfog.sum_sale_no_vat)                        as order_sum_without_vat,
         sum(qfog.is_sale_order)                          as sale_order_cnt
from dwh_data.qs_fact_order_gross qfog
left join dwh_data.dim_store ds on ds.source_store_id = qfog.source_store_id and current_date between ds.datestart and ds.dateend
where date(qfog.order_date) between to_date('2023-04-30', 'YYYY-MM-DD') and to_date('2023-05-31', 'YYYY-MM-DD')
      and qfog."source_store_id"::int > 0 and ds."store_name" like '%Торговый%'
group by qfog.source_store_id, ds.store_name
order by source_store_id;
"""

cur.execute(query)

print('Executed')

df = pd.DataFrame(cur.fetchall())

cur.close()
conn.close()

Executed


In [56]:
import psycopg2
import csv
import pandas as pd

conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                        user='rpa_robot', password='Qaz123123+')

cur = conn.cursor(name='1583_first_part')

start_date = '2023-05-01'
end_date = '2023-06-15'
store_id = '985'

query = f"""

SELECT
      q.source_store_id::int                                                                    as source_store_id,
      Sum(q."sum_order_wares_vat_nr")                                                           as "Оборот, тг с НДС",
      Sum(q."sum_order_wares_without_vat_nr")                                                   as "Оборот, тг без НДС",
      Sum(q."cost_price_base_t")                                                                as "Себестоимость",
      Sum(q."cost_price_base_t") - Sum(q."cost_price_base_t") *
         (Sum(q."sum_order_wares_vat_nr") - Sum(q."sum_order_wares_without_vat_nr"))
         / decode(Sum(q."sum_order_wares_vat_nr"), 0, null, Sum(q."sum_order_wares_vat_nr"))    as "Бухгалтерская себестоимость"
   FROM "dwh_data"."qs_fact_sales" q
   left join dwh_data.dim_store ds on ds.source_store_id = q.source_store_id and current_date between ds.datestart and ds.dateend
   WHERE date(q.order_date) between to_date('2023-04-30', 'YYYY-MM-DD') and to_date('2023-05-31', 'YYYY-MM-DD')
    and q."source_store_id"::int > 0  and ds."store_name" like '%Торговый%'
   GROUP BY
      q."source_store_id"
 union all
    SELECT
          sa.source_store_id::int                          as source_store_id,
          Sum("sum_sale")                                  as sum_order_with_vat,
          Sum(sa."sum_sale_no_vat")                        as sum_order_without_vat,
          Sum("sum_coming")                                as cost_price_with_vat ,
          Sum(sa."sum_coming_no_vat")                      as cost_price_without_vat
    FROM "dwh_data"."qs_fact_sales_gross" sa
    left join dwh_data.dim_store ds on ds.source_store_id = sa.source_store_id and current_date between ds.datestart and ds.dateend
    where date(sa.order_date) between to_date('2023-04-30', 'YYYY-MM-DD') and to_date('2023-05-31', 'YYYY-MM-DD')
   and sa.source_store_id::int > 0  and ds."store_name" like '%Торговый%'
    GROUP BY
        sa.source_store_id
    order by source_store_id;
"""

cur.execute(query)

print('Executed')

df1 = pd.DataFrame(cur.fetchall())

cur.close()
conn.close()

Executed


In [62]:
df.columns = ["Номер филиала", "Название филиала", "Количество товаров в чеке", "Выручка, тг с НДС", "Выручка, тг без НДС", "Количество чеков"]
df

,Номер филиала,Название филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,1161,6267009.92,5595544.58,47
1,4,Торговый зал АФ №1,1323651,1123343574.97,1002975175.50,169456
2,96,Торговый зал АФ №2,1467450,1337514658.00,1194201535.28,185292
3,96,Торговый зал АФ №2,362,1949488.76,1740614.95,16
4,102,Торговый зал АФ №3,300090,237329551.00,211900535.53,58198
...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,44,1098237.00,980568.75,2


In [63]:
df1.columns = ["Номер филиала", "Оборот, тг с НДС", "Оборот, тг без НДС", "Себестоимость", "Бухгалтерская себестоимость"]
df1

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,6267009.82700000000000000000000000000000038632...,5595544.48839285714285714285714285714285748778...,5925228.29737828362999639185813501223828917862...,5301183.48048092068201660021360733690452145148...
1,4,1123379963.81399999973863970000000000000000000...,1003011549.23703488965819413242857142857144278...,1030680594.30727868904444421862131647315669822...,920244772.885994759307399892562363841853388932...
2,96,1337552567.60500000022372246000000000000000000...,1194241915.85858375422253613614285714285715673...,1224889138.07348939401032913791896334478206537...,1093649690.03539375405538040957681736970528453...
3,96,1949488.74800000000000000000000000000000015639...,1740614.95357142857142857142857142857142871107...,1713925.13028708407111952111557085183239692108...,1536231.93920890884921385813890254627892582240...
4,102,237337683.542000000065436400000000000000000000...,211908646.019610158697512417857142857142860075...,227213984.802565747149723711612813502818801100...,202869629.287973827642835492024236438820882402...
...,...,...,...,...,...
279,2889,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...
280,2970,0E-42,0E-42,0E-42,None
281,2975,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...
282,2989,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...


In [64]:
df2 = df1.copy()
for i in df2['Номер филиала'].unique():
    print(df2[df2['Номер филиала'] == i].index)
    ids = df2[df2['Номер филиала'] == i].index
    print(df2.loc[ids, df2.columns[1:]].sum())
    df2.loc[ids[0], df2.columns[1:]] = df2.loc[ids, df2.columns[1:]].sum()

Index([0, 1], dtype='int64')
Оборот, тг с НДС               1129646973.640999999738639700
Оборот, тг без НДС             1008607093.725427746801051275
Себестоимость                  1036605822.604656972674440610
Бухгалтерская себестоимость    925545956.3664756799894164928
dtype: object
Index([2, 3], dtype='int64')
Оборот, тг с НДС               1339502056.353000000223722460
Оборот, тг без НДС             1195982530.812155182793964708
Себестоимость                  1226603063.203776478081448659
Бухгалтерская себестоимость    1095185921.974602662904594268
dtype: object
Index([4, 5], dtype='int64')
Оборот, тг с НДС               240085507.5420000000654364000
Оборот, тг без НДС             214362060.3053244444117981321
Себестоимость                  229709513.5536203471497237116
Бухгалтерская себестоимость    205101425.3695876276428354920
dtype: object
Index([6, 7], dtype='int64')
Оборот, тг с НДС               578929044.2019999998617525300
Оборот, тг без НДС             516899921.40458887

In [65]:
df1

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,6267009.82700000000000000000000000000000038632...,5595544.48839285714285714285714285714285748778...,5925228.29737828362999639185813501223828917862...,5301183.48048092068201660021360733690452145148...
1,4,1123379963.81399999973863970000000000000000000...,1003011549.23703488965819413242857142857144278...,1030680594.30727868904444421862131647315669822...,920244772.885994759307399892562363841853388932...
2,96,1337552567.60500000022372246000000000000000000...,1194241915.85858375422253613614285714285715673...,1224889138.07348939401032913791896334478206537...,1093649690.03539375405538040957681736970528453...
3,96,1949488.74800000000000000000000000000000015639...,1740614.95357142857142857142857142857142871107...,1713925.13028708407111952111557085183239692108...,1536231.93920890884921385813890254627892582240...
4,102,237337683.542000000065436400000000000000000000...,211908646.019610158697512417857142857142860075...,227213984.802565747149723711612813502818801100...,202869629.287973827642835492024236438820882402...
...,...,...,...,...,...
279,2889,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...
280,2970,0E-42,0E-42,0E-42,None
281,2975,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...
282,2989,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...


In [66]:
df2

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,1129646973.640999999738639700,1008607093.725427746801051275,1036605822.604656972674440610,925545956.3664756799894164928
1,4,1123379963.81399999973863970000000000000000000...,1003011549.23703488965819413242857142857144278...,1030680594.30727868904444421862131647315669822...,920244772.885994759307399892562363841853388932...
2,96,1339502056.353000000223722460,1195982530.812155182793964708,1226603063.203776478081448659,1095185921.974602662904594268
3,96,1949488.74800000000000000000000000000000015639...,1740614.95357142857142857142857142857142871107...,1713925.13028708407111952111557085183239692108...,1536231.93920890884921385813890254627892582240...
4,102,240085507.5420000000654364000,214362060.3053244444117981321,229709513.5536203471497237116,205101425.3695876276428354920
...,...,...,...,...,...
279,2889,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...
280,2970,0E-42,0E-42,0E-42,0
281,2975,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...
282,2989,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...


In [67]:
df2 = df2.loc[~df2['Номер филиала'].duplicated(keep='first')]

In [68]:
df2

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,1129646973.640999999738639700,1008607093.725427746801051275,1036605822.604656972674440610,925545956.3664756799894164928
2,96,1339502056.353000000223722460,1195982530.812155182793964708,1226603063.203776478081448659,1095185921.974602662904594268
4,102,240085507.5420000000654364000,214362060.3053244444117981321,229709513.5536203471497237116,205101425.3695876276428354920
6,105,578929044.2019999998617525300,516899921.4045888748025048478,537354539.5527312199941891141,479786066.7917906438582291571
8,116,490101131.4837000005979525100,437708737.3946928404883147482,478358695.8100691511677571746,427221655.2541790727437284717
...,...,...,...,...,...
279,2889,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...
280,2970,0E-42,0E-42,0E-42,0
281,2975,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...
282,2989,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...


In [69]:
len(df2)

241

In [80]:
df3 = df.copy()
for i in df3['Номер филиала'].unique():
    print(i, df3[df3['Номер филиала'] == i].index)
    ids = df3[df3['Номер филиала'] == i].index
    print(df3.loc[ids, df3.columns[2:]].sum())
    df3.loc[ids[0], df3.columns[2:]] = df3.loc[ids, df3.columns[1:]].sum()
    try:
        df3 = df3.drop([ids[1]])
    except:
        pass

4 Index([0, 1], dtype='int64')
Количество товаров в чеке          1324812
Выручка, тг с НДС            1129610584.89
Выручка, тг без НДС          1008570720.08
Количество чеков                    169503
dtype: object
96 Index([2, 3], dtype='int64')
Количество товаров в чеке          1467812
Выручка, тг с НДС            1339464146.76
Выручка, тг без НДС          1195942150.23
Количество чеков                    185308
dtype: object
102 Index([4, 5], dtype='int64')
Количество товаров в чеке          300259
Выручка, тг с НДС            240077375.00
Выручка, тг без НДС          214353949.81
Количество чеков                    58213
dtype: object
105 Index([6, 7], dtype='int64')
Количество товаров в чеке          712262
Выручка, тг с НДС            578917761.74
Выручка, тг без НДС          516888589.54
Количество чеков                   104683
dtype: object
116 Index([8, 9], dtype='int64')
Количество товаров в чеке                           375356
Выручка, тг с НДС            490099440.1100

In [81]:
df3

,Номер филиала,Название филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,1324812,1129610584.89,1008570720.08,169503
2,96,Торговый зал АФ №2,1467812,1339464146.76,1195942150.23,185308
4,102,Торговый зал АФ №3,300259,240077375.00,214353949.81,58213
6,105,Торговый зал КФ №1,712262,578917761.74,516888589.54,104683
8,116,Торговый зал АСФ №1,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,44,1098237.00,980568.75,2


In [74]:
df

,Номер филиала,Название филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,1161,6267009.92,5595544.58,47
1,4,Торговый зал АФ №1,1323651,1123343574.97,1002975175.50,169456
2,96,Торговый зал АФ №2,1467450,1337514658.00,1194201535.28,185292
3,96,Торговый зал АФ №2,362,1949488.76,1740614.95,16
4,102,Торговый зал АФ №3,300090,237329551.00,211900535.53,58198
...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,44,1098237.00,980568.75,2


In [72]:
df3

,Номер филиала,Название филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1Торговый зал АФ №1,1324812,1129610584.89,1008570720.08,169503
2,96,Торговый зал АФ №2Торговый зал АФ №2,1467812,1339464146.76,1195942150.23,185308
4,102,Торговый зал АФ №3Торговый зал АФ №3,300259,240077375.00,214353949.81,58213
6,105,Торговый зал КФ №1Торговый зал КФ №1,712262,578917761.74,516888589.54,104683
8,116,Торговый зал АСФ №1Торговый зал АСФ №1,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,44,1098237.00,980568.75,2


In [83]:
df4 = pd.concat([df2, df3], join='inner', axis=1)
df4.columns = ['Номер филиала', 'Оборот, тг с НДС', 'Оборот, тг без НДС',
       'Себестоимость', 'Бухгалтерская себестоимость', 'Номер филиала1',
       'Название филиала', 'Количество товаров в чеке', 'Выручка, тг с НДС',
       'Выручка, тг без НДС', 'Количество чеков']
df4.insert(1, 'Название филиала', df4.pop('Название филиала'))
df4

,Номер филиала,Название филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость,Номер филиала1,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,1129646973.640999999738639700,1008607093.725427746801051275,1036605822.604656972674440610,925545956.3664756799894164928,4,1324812,1129610584.89,1008570720.08,169503
2,96,Торговый зал АФ №2,1339502056.353000000223722460,1195982530.812155182793964708,1226603063.203776478081448659,1095185921.974602662904594268,96,1467812,1339464146.76,1195942150.23,185308
4,102,Торговый зал АФ №3,240085507.5420000000654364000,214362060.3053244444117981321,229709513.5536203471497237116,205101425.3695876276428354920,102,300259,240077375.00,214353949.81,58213
6,105,Торговый зал КФ №1,578929044.2019999998617525300,516899921.4045888748025048478,537354539.5527312199941891141,479786066.7917906438582291571,105,712262,578917761.74,516888589.54,104683
8,116,Торговый зал АСФ №1,490101131.4837000005979525100,437708737.3946928404883147482,478358695.8100691511677571746,427221655.2541790727437284717,116,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...,2889,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,0E-42,0E-42,0E-42,0,2970,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...,2975,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,2989,44,1098237.00,980568.75,2


In [84]:
df4 = df4.drop(['Номер филиала1'], axis=1)
df4

,Номер филиала,Название филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,1129646973.640999999738639700,1008607093.725427746801051275,1036605822.604656972674440610,925545956.3664756799894164928,1324812,1129610584.89,1008570720.08,169503
2,96,Торговый зал АФ №2,1339502056.353000000223722460,1195982530.812155182793964708,1226603063.203776478081448659,1095185921.974602662904594268,1467812,1339464146.76,1195942150.23,185308
4,102,Торговый зал АФ №3,240085507.5420000000654364000,214362060.3053244444117981321,229709513.5536203471497237116,205101425.3695876276428354920,300259,240077375.00,214353949.81,58213
6,105,Торговый зал КФ №1,578929044.2019999998617525300,516899921.4045888748025048478,537354539.5527312199941891141,479786066.7917906438582291571,712262,578917761.74,516888589.54,104683
8,116,Торговый зал АСФ №1,490101131.4837000005979525100,437708737.3946928404883147482,478358695.8100691511677571746,427221655.2541790727437284717,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,0E-42,0E-42,0E-42,0,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,44,1098237.00,980568.75,2


In [148]:
import openpyxl
import os
import datetime

for i in range(30, len(df4)):
    book = openpyxl.load_workbook('samples\\2023-04-30_2023-05-31_1583_4.xlsx')
    sheet = book.active
    
    try:
        start_date = start_date.split('-')[2] + '.' + start_date.split('-')[1] + '.' + start_date.split('-')[0]
        end_date = end_date.split('-')[2] + '.' + end_date.split('-')[1] + '.' + end_date.split('-')[0]
    except:
        ...

    sheet['D2'] = start_date
    sheet['G2'] = end_date
    sheet['B4'].value = str(datetime.datetime.now().strftime("%d.%m.%y %H:%M:%S"))
    sheet['B7'].value = df4['Название филиала'].iloc[i]
    sheet['A11'].value = start_date + '-' + end_date
    sheet['B11'].value = df4['Название филиала'].iloc[i]
    sheet['C11'].value = df4['Название филиала'].iloc[i]
    sheet['D11'].value = df4['Количество чеков'].iloc[i]
    sheet['E11'].value = df4['Выручка, тг с НДС'].iloc[i]
    sheet['F11'].value = df4['Выручка, тг без НДС'].iloc[i]
    sheet['G11'].value = df4['Оборот, тг с НДС'].iloc[i]
    sheet['H11'].value = df4['Оборот, тг без НДС'].iloc[i]
    sheet['I11'].value = df4['Себестоимость'].iloc[i]
    sheet['J11'].value = df4['Бухгалтерская себестоимость'].iloc[i]
    sheet['K11'].value = float(df4['Оборот, тг с НДС'].iloc[i]) - float(df4['Себестоимость'].iloc[i])
    try:
        sheet['L11'].value = float(df4['Оборот, тг с НДС'].iloc[i]) / float(df4['Себестоимость'].iloc[i]) - 1
    except:
        sheet['L11'].value = 0
    sheet['L11'].number_format = '0.00%'
    sheet['M11'].value = df4['Количество товаров в чеке'].iloc[i]

    sheet['D12'].value = sheet['D11'].value
    sheet['E12'].value = sheet['E11'].value
    sheet['F12'].value = sheet['F11'].value
    sheet['G12'].value = sheet['G11'].value
    sheet['H12'].value = sheet['H11'].value
    sheet['I12'].value = sheet['I11'].value
    sheet['J12'].value = sheet['J11'].value
    sheet['K12'].value = sheet['K11'].value
    sheet['L12'].value = sheet['L11'].value
    sheet['M12'].value = sheet['M11'].value
    
    try:
        os.makedirs('samples\\1583')
    except:
        pass
    book.save(f'samples\\1583\\{df4["Название филиала"].iloc[i]}_1583.xlsx')
    print('Saved', df4['Название филиала'].iloc[i])
    break

Saved Торговый зал АФ №21


In [133]:
df4


,Номер филиала,Название филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,1129646973.640999999738639700,1008607093.725427746801051275,1036605822.604656972674440610,925545956.3664756799894164928,1324812,1129610584.89,1008570720.08,169503
2,96,Торговый зал АФ №2,1339502056.353000000223722460,1195982530.812155182793964708,1226603063.203776478081448659,1095185921.974602662904594268,1467812,1339464146.76,1195942150.23,185308
4,102,Торговый зал АФ №3,240085507.5420000000654364000,214362060.3053244444117981321,229709513.5536203471497237116,205101425.3695876276428354920,300259,240077375.00,214353949.81,58213
6,105,Торговый зал КФ №1,578929044.2019999998617525300,516899921.4045888748025048478,537354539.5527312199941891141,479786066.7917906438582291571,712262,578917761.74,516888589.54,104683
8,116,Торговый зал АСФ №1,490101131.4837000005979525100,437708737.3946928404883147482,478358695.8100691511677571746,427221655.2541790727437284717,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,0E-42,0E-42,0E-42,0,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,44,1098237.00,980568.75,2


In [131]:
book.save('samples\\kekus.xlsx')

In [100]:
df4

,Номер филиала,Название филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,1129646973.640999999738639700,1008607093.725427746801051275,1036605822.604656972674440610,925545956.3664756799894164928,1324812,1129610584.89,1008570720.08,169503
2,96,Торговый зал АФ №2,1339502056.353000000223722460,1195982530.812155182793964708,1226603063.203776478081448659,1095185921.974602662904594268,1467812,1339464146.76,1195942150.23,185308
4,102,Торговый зал АФ №3,240085507.5420000000654364000,214362060.3053244444117981321,229709513.5536203471497237116,205101425.3695876276428354920,300259,240077375.00,214353949.81,58213
6,105,Торговый зал КФ №1,578929044.2019999998617525300,516899921.4045888748025048478,537354539.5527312199941891141,479786066.7917906438582291571,712262,578917761.74,516888589.54,104683
8,116,Торговый зал АСФ №1,490101131.4837000005979525100,437708737.3946928404883147482,478358695.8100691511677571746,427221655.2541790727437284717,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,0E-42,0E-42,0E-42,0,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,44,1098237.00,980568.75,2


In [ ]:
for i in df2['Номер филиала'].unique():
    print(df2[df2['Номер филиала'] == i].index)
    ids = df2[df2['Номер филиала'] == i].index
    print(df2.loc[ids, df2.columns[1:]].sum())
    df2.loc[ids[0], df2.columns[1:]] = df2.loc[ids, df2.columns[1:]].sum()

Index([0], dtype='int64')
Количество товаров в чеке                                                  11783
Выручка, тг с НДС                                                     7525031.00
Выручка, тг без НДС                                                   6718734.66
Количество чеков                                                            2589
Оборот, тг с НДС               82133802811.8219999545656379960000000000000000...
Оборот, тг без НДС             73347493095.2100944447412386556031746031742836...
Себестоимость                  57494025081.6455697745459859511759702966979874...
Бухгалтерская себестоимость    51343569435.7603851262871592390984197815968943...
dtype: object
Index([1, 2], dtype='int64')
Количество товаров в чеке                          132801427
Выручка, тг с НДС              82365149978.53000000008417696
Выручка, тг без НДС            73521297372.96154761912279914
Количество чеков                                    13344863
Оборот, тг с НДС               57053341

In [ ]:
df2.loc[[1, 2], df2.columns[1:]].sum()

Количество товаров в чеке                          132875366
Выручка, тг с НДС              82596248661.23000000016835392
Выручка, тг без НДС            73727900744.35309523824559828
Количество чеков                                    13348164
Оборот, тг с НДС               578058850.0393632132343045094
Оборот, тг без НДС             516389234.9938097469773039315
Себестоимость                  554449338.2460351552897313918
Бухгалтерская себестоимость    498882079.7768801754627076399
dtype: object

In [ ]:
pd.concat([df, df1], axis=1, join="inner")

,Номер филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,985,11783,7525031.00,6718734.66,2589,82133802811.8219999545656379960000000000000000...,73347493095.2100944447412386556031746031742836...,57494025081.6455697745459859511759702966979874...,51343569435.7603851262871592390984197815968943...
1,4,132727488,82134051295.83,73314694001.57,13341562,563007982.993363213274250389408395786066240549...,502950960.845595461489249895701940682003587788...,540527736.599755072089731391755183500350775607...,486452078.306987244210403794698508562867111343...
2,4,73939,231098682.700000000084176960000000000000000067...,206603371.391547619122799142857142857142857202...,3301,7525433.52299999998002706000000000000000000000...,6719137.07410714274402701785714285714285733855...,6960800.82314004159999999999999999999999999200...,6215000.73494646562615192260393837401195928088...


In [46]:
for i in df3['Номер филиала']:
    print(i, end=', ')

4, 96, 102, 105, 116, 140, 197, 220, 280, 320, 360, 401, 421, 441, 444, 461, 501, 521, 541, 542, 561, 601, 602, 631, 632, 633, 634, 635, 641, 681, 701, 721, 725, 767, 785, 805, 832, 845, 849, 852, 866, 885, 889, 893, 897, 901, 925, 945, 949, 953, 957, 961, 965, 969, 985, 989, 993, 1005, 1009, 1025, 1029, 1033, 1069, 1105, 1135, 1136, 1137, 1138, 1146, 1151, 1152, 1165, 1187, 1205, 1208, 1373, 1375, 1412, 1432, 1435, 1439, 1452, 1552, 1555, 1556, 1557, 1565, 1568, 1571, 1574, 1578, 1581, 1584, 1592, 1595, 1615, 1638, 1698, 1718, 1738, 1778, 1782, 1798, 1818, 1838, 1841, 1858, 1861, 1864, 1867, 1870, 1883, 1886, 1898, 1907, 1910, 1913, 1938, 1941, 1944, 1947, 1950, 1955, 1956, 1959, 1960, 1963, 1966, 1970, 1975, 1979, 1980, 1998, 2018, 2024, 2027, 2029, 2032, 2041, 2044, 2047, 2050, 2059, 2061, 2065, 2071, 2074, 2078, 2081, 2083, 2087, 2091, 2098, 2099, 2104, 2110, 2118, 2121, 2124, 2127, 2138, 2141, 2142, 2147, 2150, 2151, 2152, 2155, 2158, 2161, 2162, 2167, 2181, 2182, 2183, 2184, 2193

In [8]:
s = [4, 96, 102, 105, 116, 140, 197, 220, 280, 320, 360, 401, 421, 441, 444, 461, 501, 521, 541, 542, 561, 601, 602, 631, 632, 633, 634, 635, 641, 681, 701, 721, 725, 767, 785, 805, 832, 845, 849, 852, 866, 885, 889, 893, 897, 901, 925, 945, 949, 953, 957, 961, 965, 969, 985, 989, 993, 1005, 1009, 1025, 1029, 1033, 1065, 1069, 1105, 1135, 1136, 1137, 1138, 1146, 1151, 1152, 1187, 1205, 1208, 1332, 1373, 1375, 1412, 1432, 1435, 1439, 1452, 1552, 1555, 1556, 1557, 1565, 1568, 1571, 1574, 1578, 1581, 1584, 1592, 1595, 1612, 1615, 1638, 1698, 1738, 1778, 1782, 1798, 1818, 1838, 1841, 1858, 1861, 1864, 1867, 1870, 1879, 1883, 1898, 1907, 1910, 1913, 1918, 1938, 1941, 1944, 1947, 1950, 1955, 1956, 1959, 1960, 1963, 1966, 1970, 1975, 1979, 1980, 1998, 2018, 2019, 2024, 2026, 2027, 2029, 2032, 2041, 2044, 2047, 2050, 2059, 2061, 2065, 2066, 2071, 2074, 2078, 2081, 2083, 2087, 2091, 2098, 2099, 2104, 2110, 2118, 2121, 2124, 2127, 2138, 2141, 2142, 2147, 2150, 2151, 2152, 2155, 2158, 2161, 2162, 2167, 2181, 2182, 2183, 2184, 2193, 2196, 2241, 2261, 2262, 2281, 2290, 2294, 2299, 2300, 2303, 2304, 2305, 2306, 2315, 2320, 2323, 2326, 2341, 2344, 2347, 2350, 2353, 2361, 2364, 2367, 2381, 2382, 2383, 2384, 2401, 2402, 2403, 2412, 2421, 2422, 2423, 2424, 2433, 2436, 2439, 2442, 2445, 2482, 2485, 2488, 2491, 2494, 2521, 2524, 2541, 2563, 2583, 2601, 2604, 2641, 2643, 2655, 2661, 2684, 2702, 2723, 2741, 2745, 2748, 2751, 2763, 2766, 2781, 2785, 2801, 2809, 2810, 2829, 2832, 2849, 2889, 2970, 2975, 2989, 3012, 3033]
s1 = [4, 96, 102, 105, 116, 140, 197, 220, 280, 320, 360, 401, 421, 441, 444, 461, 501, 521, 541, 542, 561, 601, 602, 631, 632, 633, 634, 635, 641, 681, 701, 721, 725, 767, 785, 805, 832, 845, 849, 852, 866, 885, 889, 893, 897, 901, 925, 945, 949, 953, 957, 961, 965, 969, 985, 989, 993, 1005, 1009, 1025, 1029, 1033, 1069, 1105, 1135, 1136, 1137, 1138, 1146, 1151, 1152, 1165, 1187, 1205, 1208, 1373, 1375, 1412, 1432, 1435, 1439, 1452, 1552, 1555, 1556, 1557, 1565, 1568, 1571, 1574, 1578, 1581, 1584, 1592, 1595, 1615, 1638, 1698, 1718, 1738, 1778, 1782, 1798, 1818, 1838, 1841, 1858, 1861, 1864, 1867, 1870, 1883, 1886, 1898, 1907, 1910, 1913, 1938, 1941, 1944, 1947, 1950, 1955, 1956, 1959, 1960, 1963, 1966, 1970, 1975, 1979, 1980, 1998, 2018, 2024, 2027, 2029, 2032, 2041, 2044, 2047, 2050, 2059, 2061, 2065, 2071, 2074, 2078, 2081, 2083, 2087, 2091, 2098, 2099, 2104, 2110, 2118, 2121, 2124, 2127, 2138, 2141, 2142, 2147, 2150, 2151, 2152, 2155, 2158, 2161, 2162, 2167, 2181, 2182, 2183, 2184, 2193, 2196, 2241, 2261, 2262, 2281, 2290, 2294, 2299, 2300, 2303, 2304, 2305, 2306, 2315, 2320, 2323, 2326, 2341, 2344, 2347, 2350, 2353, 2361, 2364, 2367, 2381, 2382, 2383, 2384, 2401, 2402, 2403, 2412, 2421, 2422, 2423, 2433, 2436, 2439, 2501, 2521, 2524, 2541, 2563, 2582, 2601, 2604, 2641, 2643, 2655, 2661, 2684, 2702, 2723, 2741, 2745, 2751, 2763, 2781, 2785, 2801, 2810, 2829, 2849, 2869, 2889, 2970, 2975, 2989, 3012]
len(s)

263

In [64]:
for i in s:
    print(i)

4
96
102
105
116
140
197
220
280
320
360
401
421
441
444
461
501
521
541
542
561
601
602
631
632
633
634
635
641
681
701
721
725
767
785
805
832
845
849
852
866
885
889
893
897
901
925
945
949
953
957
961
965
969
985
989
993
1005
1009
1025
1029
1033
1065
1069
1105
1135
1136
1137
1138
1146
1151
1152
1187
1205
1208
1332
1373
1375
1412
1432
1435
1439
1452
1552
1555
1556
1557
1565
1568
1571
1574
1578
1581
1584
1592
1595
1612
1615
1638
1698
1738
1778
1782
1798
1818
1838
1841
1858
1861
1864
1867
1870
1879
1883
1898
1907
1910
1913
1918
1938
1941
1944
1947
1950
1955
1956
1959
1960
1963
1966
1970
1975
1979
1980
1998
2018
2019
2024
2026
2027
2029
2032
2041
2044
2047
2050
2059
2061
2065
2066
2071
2074
2078
2081
2083
2087
2091
2098
2099
2104
2110
2118
2121
2124
2127
2138
2141
2142
2147
2150
2151
2152
2155
2158
2161
2162
2167
2181
2182
2183
2184
2193
2196
2241
2261
2262
2281
2290
2294
2299
2300
2303
2304
2305
2306
2315
2320
2323
2326
2341
2344
2347
2350
2353
2361
2364
2367
2381
2382
2383
2384
2401


In [9]:
import numpy as np
len(np.unique(np.asarray(s)))

263

In [10]:
s = np.asarray(s)
s1 = np.asarray(s1)
print(np.setdiff1d(s, s1))

[1065 1332 1612 1879 1918 2019 2026 2066 2424 2442 2445 2482 2485 2488
 2491 2494 2583 2748 2766 2809 2832 3033]


In [51]:
len(df3)

247